### Import packages

In [ ]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

### Extract data from csv

In [ ]:
raw_data=np.loadtxt('Audiobooks-data.csv',delimiter=',')

unscaled_inputs=raw_data[:,1:-1]

targets=raw_data[:,-1]

### Balancing the dataset

In [ ]:
num_one_targets=int(np.sum(targets))
zero_targets_counter=0

indices_to_remove=[]
for i in range (targets.shape[0]):
    if targets[i]==0:
        zero_targets_counter+=1
        if zero_targets_counter>num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_all=np.delete(unscaled_inputs,indices_to_remove,axis=0)
targets_equal_all=np.delete(targets,indices_to_remove,axis=0)

### Standardize the inputs

In [ ]:
scaled_inputs=preprocessing.scale(unscaled_inputs_all)

### Shuffle the data

In [ ]:
shuffled_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=targets_equal_all[shuffled_indices]

### Split the data in training,validation,test

In [ ]:
samples_count=shuffled_inputs.shape[0]
training_samples_count=int(0.8*samples_count)
validation_samples_count=int(0.1*samples_count)
test_samples_count=samples_count-training_samples_count-validation_samples_count

In [ ]:
train_inputs=shuffled_inputs[:training_samples_count]
train_targets=shuffled_targets[:training_samples_count]

validation_inputs=shuffled_inputs[training_samples_count:training_samples_count+validation_samples_count]
validation_targets=shuffled_targets[training_samples_count:training_samples_count+validation_samples_count]

test_inputs=shuffled_inputs[training_samples_count+validation_samples_count:]
test_targets=shuffled_targets[training_samples_count+validation_samples_count:]

In [ ]:
print((np.sum(train_targets))/training_samples_count)
print((np.sum(validation_targets))/validation_samples_count)
print((np.sum(test_targets))/test_samples_count)

### Save in npz

In [ ]:
np.savez('Audiobooks_train',inputs=train_inputs,targets=train_targets)
np.savez('Audiobooks_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audiobooks_test',inputs=test_inputs,targets=test_targets)

In [ ]:
train_inputs=train_inputs.astype(np.float)
train_targets=train_targets.astype(np.int)

In [ ]:
validation_inputs=validation_inputs.astype(np.float)
validation_targets=validation_targets.astype(np.int)

In [ ]:
test_inputs=test_inputs.astype(np.float)
test_targets=test_targets.astype(np.int)

### Model

In [ ]:
input_size=10
output_size=2
hidden_layer_size=100

model=tf.keras.Sequential([
        tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
        tf.keras.layers.Dense(output_size,activation='softmax')
    
    
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [ ]:
batch_size=100
max_epochs=100

early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs,validation_targets),
          verbose=2
         )

### Test

In [ ]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_targets)